In [1]:
import numpy as np
import time, os, sys, random
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import nd2reader
import pims
import xml.etree.ElementTree as ET
from IPython.display import clear_output
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

In [2]:
from cellpose import models, utils

In [3]:
eDir = ""
imgDir = "raw/"
maskDir = "mask/"
ssTabFile = eDir + "slideset-table.xml"

ssTab = ET.parse(ssTabFile)
imageFiles = [e.text for e in ssTab.find("./col[@name='Img']").findall("e")]
imageNames = list(map(lambda f : os.path.basename(f), imageFiles))

chan = (1,2)
diam = 400

In [4]:
def runTheModel(imageName):
    clear_output(wait=True)
    print("Now working on: " + imageName)
    fname = eDir + imgDir + imageName
    fnameOut = eDir + maskDir + imageName

    # Load and pre-process the image (may need to adjust this depending on image format)
    I = pims.Bioformats(fname)
    I.bundle_axes = 'yxc'
    img = I[0]
    img2 = np.zeros_like(img[:,:,[0,1]], dtype=float)
    img2[:,:,0] = img[:,:,1] / np.linalg.norm(img[:,:,1]) + \
        img[:,:,2] / np.linalg.norm(img[:,:,2]) + \
        img[:,:,3] / np.linalg.norm(img[:,:,3]) # Sum K5-mNG, K6A-mRb, and PLA
    img2[:,:,1] = img[:,:,0] # DAPI
    
    # Run the Cellpose model
    model = models.Cellpose(gpu=False, model_type='cyto')
    masks, flows, styles, diams = model.eval(img2, diameter=diam, channels=(1,2))
    
    # Save the masks
    maskout = fnameOut + '_masks-' + str(diam) + '.png'
    skimage.io.imsave(maskout, masks, check_contrast=False)

In [ ]:
for i in imageNames:
    runTheModel(i)